In [ ]:
using LinearAlgebra
using Plots

# <span style="color:MediumSlateBlue;font-weight:bold">Exercice 1 : Puissance itérée</span>

Soit $A \in \mathbb{M}_{2,2}$ une matrice sous la forme 
$$
    A = P D P^t, 
$$
où $P$ est une matrice orthogonale (ou unitaire (telle que $P^{-1} = P^t$)), et $D = \begin{pmatrix} 1 & 0 \\ 0 & \lambda \end{pmatrix}$ est une matrice diagonale avec $0 \leqslant \lambda < 1$. On note $p_1$ et $p_2$ les deux colonnes de $P$.

1. Complétez le code ci-dessous, qui affiche $p_1$, $p_2$, et pour un vecteur $v$ quelconque, la suite $v$, $A v$, $A^2 v$, etc... Qu'observez-vous ? Donnez une justification mathématique.

In [ ]:
## Construction d'une matrice P orthogonale
p1 = 2.0 .* (rand(2) .- 0.5) # vous pouvez le fixer si vous voulez
p1 ./= norm(p1)
p2 = [p1[2],-p1[1]]
P = [p1[1] p2[1]; p1[2] p2[2]]

## Construction de la matrice D 
λ = 0.5 
D = [1.0 0.0; 0.0 λ]

A = P * D * P'

v = 2.0 .* (rand(2) .- 0.5)
depth = 5 # nombre d'itérations

function plot_arrow!(p, arrow; color=:blue, tag="", lw=1)
    plot!(p, [0.0,arrow[1]], [0.0,arrow[2]], lw=lw, arrow=true, color=color)
    if length(tag) > 0
        annotate!(p, arrow[1]+0.1, arrow[2], text(tag, color))
    end
end

p = plot(xlims=[-1.2,1.2], ylims=[-1.2,1.2], legend=false, aspect_ratio=:equal)
plot_arrow!(p, p1; tag="p₁", color=:purple, lw=2)
plot_arrow!(p, p2; tag="p₂", color=:purple, lw=2)

# Ici votre code : on vous demande d'afficher le vecteur v, puis A * v, puis A^2 * v, etc jusqu'a A^depth * v
# plot_arrow!(p, v; tag="v")

display(p)

2. On cherche maintenant à exploiter ce comportement pour reconstruire $p_1$ à partir d'une matrice $A$ donnée. Proposez et implémentez une méthode pour ce faire. 

In [ ]:
function get_p1(A::Matrix{Float64})::Vector{Float64}
    init = 2.0 .* (rand(2) .- 0.5)
    depth = 20
    for idepth in 1:depth 
        # ... ici votre code
    end
    return # ...
end

hatp1 = get_p1(A)
println("p1 : ", p1, ", approximation : ", hatp1)
println("Erreur sur la direction : ", min(norm(p1 .- hatp1), norm(p1 .+ hatp1)))

3. Une fois la direction $p_1$ connue, vous pouvez en déduire la direction $p_2$ par orthogonalité. Comment obtenir une approximation des valeurs propres ? Déduisez-en une fonction donnant une approximation des éléments propres d'une matrice symétrique positive en dimension 2. 

In [ ]:
function approx_diagonalisation(A::Matrix{Float64})::Tuple{Vector{Float64},Vector{Float64},Float64,Float64}
    p1 = get_p1(A)
    p2 = # ... 
    λ1 = # ...
    λ2 = # ...
    return p1, p2, λ1, λ2
end

# Attention, l'algorithme ci-dessus ne marche que pour les matrices dont les valeurs propres sont suffisamment distinctes. 
# Donc ne testez pas avec la matrice identité

hatp1, hatp2, hatλ1, hatλ2 = approx_diagonalisation(A)
println("Approximation : ")
println("Premier vecteur propre ", hatp1, ", valeur propre ", hatλ1)
println("Second  vecteur propre ", hatp2, ", valeur propre ", hatλ2)
hatP = [hatp1[1] hatp2[1]; hatp1[2] hatp2[2]]
println("Erreur sur la reconstruction de A : ", norm(A .- hatP * [hatλ1 0.0; 0.0 hatλ2] * hatP'))

# Bonus : comment pourriez-vous généraliser cet algorithme à la dimension 3 ?

# <span style="color:MediumSlateBlue;font-weight:bold">Exercice 2 : Décomposition en valeurs singulières</span>

Le but de cet exercice est de suivre les étapes de la décomposition en valeurs singulières, en les programmant. On travaillera sur des matrices de rang 2 pour pouvoir utiliser la fonction `approx_diagonalisation` de l'exercice précédent.

1. __Étape 1__ : La fonction suivante permute les colonnes de $A$ pour que les deux premières soient linéairement indépendantes. Elle garde en mémoire la permutation sous forme de matrice $\Pi \in \mathbb{M}_{m,m}$. Testez ce code sur les exemples fournis pour pouvoir l'utiliser après.

In [ ]:
function permute_columns(A::Matrix{Float64})::Tuple{Matrix{Float64},Matrix{Float64}} # renvoie A permutée et Π
    Aperm = copy(A)
    m = size(A)[2]
    Π = collect(I(m))
    # Trouver la première colonne de norme > 0
    normes_colonnes = [norm(A[:,i]) for i in 1:m]
    index_first = findfirst(normes_colonnes .> 1e-6)
    if index_first != 1
        # permutation des colonnes de A
        sauvegarde = copy(Aperm[:,1])
        Aperm[:,1] .= Aperm[:,index_first]
        Aperm[:,index_first] .= sauvegarde
        # enregistrement dans la matrice de permutation 
        sauvegarde = copy(Π[:,1])
        Π[:,1] .= Π[:,index_first]
        Π[:,index_first] .= sauvegarde
    end
    # Trouver la première colonne linéairement indépendante 
    # i.e. la première formant un angle strictement entre 0 et π avec la première colonne 
    # donc telle que cos(angle(q,p)) = <q,p> / (|p| |q|) ∈ ]-1,1[.
    is_indep = [(norm(Aperm[:,i]) > 1e-6) && (abs((Aperm[:,i]' * Aperm[:,1]) / (norm(Aperm[:,1]) * norm(Aperm[:,i]))) < 1.0-1e-6) for i in 1:m]
    index_second = findfirst(is_indep)
    if index_second != 2
        # permutation des colonnes de A
        sauvegarde = copy(Aperm[:,2])
        Aperm[:,2] .= Aperm[:,index_second]
        Aperm[:,index_second] .= sauvegarde
        # enregistrement dans la matrice de permutation 
        sauvegarde = copy(Π[:,2])
        Π[:,2] .= Π[:,index_second]
        Π[:,index_second] .= sauvegarde
    end
    return Aperm, Π
end

# Testez avec les exemples suivants, et ceux que vous pourrez imaginer
# A = [1.0 0.0 1.0 1.0 0.0; 0.0 1.0 0.0 1.0 1.0] # déjà dans le bon ordre 
# A = [0.0 1.0 0.0; 0.0 0.0 1.0; 0.0 0.0 0.1] # première colonne nulle
# A = [0.0 0.0 0.0 1.0 0.5; 1.0 0.5 0.0 0.0 0.3] # première colonne bien placée, 4e colonne est celle voulue en 2e position
# A = [0.0 0.0 0.0 1.0 0.5; 0.0 0.5 0.0 0.0 0.3] # première colonne mal placée, 4e colonne est celle voulue en 2e position

Aperm, Π = permute_columns(A)
display(A)
display(Aperm)
display(Π)
display(A * Π)

2. __Étape 2__ : Pour s'éviter les répétitions, on code une fonction qui fera les deux factorisations QR dont nous aurons besoin. Implémentez la fonction ci-dessous, qui prend en entrée une matrice rectangulaire $A \in \mathbb{M}_{\ell,c}$ dont on sait 
- qu'elle a au moins deux colonnes, 
- que les deux premières sont linéairement indépendantes, 
et renvoie des matrices $Q \in \mathbb{R}_{\ell,2}$ et $R \in \mathbb{M}_{2,c}$ telles que $A = Q R$, la matrice $Q$ satisfait $Q^t Q = \mathbb{I}_2$, et $R_{2,1} = 0$.

In [ ]:
function debut_QR(A::Matrix{Float64})::Tuple{Matrix{Float64},Matrix{Float64}}
    (l,c) = size(A)
    Q = zeros(l,2)
    R = zeros(2,c)

    # ici votre code 

    return Q, R
end

Q, R = debut_QR(Aperm)
println("Erreur entre Aperm et Q*R : ", norm(Aperm .- Q*R))
display(Aperm)
display(Q)
display(R)

3. Complétez l'étape 2 en décomposant $A_{\text{perm}}$ sous la forme $Q Sᵗ Tᵗ$, d'abord en appliquant la fonction précédente à $A_{\text{perm}}$ pour obtenir $Q R$, puis à $Rᵗ$ pour obtenir $S$ et $T$. 

In [ ]:
function get_QST(A::Matrix{Float64})::Tuple{Matrix{Float64},Matrix{Float64},Matrix{Float64}}
    Q, R = # ...
    Rtranspose = collect(R')
    T, S = # ...
    return Q, S, T 
end

Q, S, T = get_QST(Aperm)
println("Erreur entre Aperm et Q * S' * T' : ", norm(Aperm .- Q * S' * T'))

4. __Étape 3__ : Utilisez le code de l'exercice précédent pour diagonaliser $S S^{t}$ en $P D P^{t}$. En déduire la matrice $\Sigma = \sqrt{D}$.  

In [ ]:
function get_PΣ(S::Matrix{Float64})::Tuple{Matrix{Float64},Matrix{Float64}} # sorties : P, Σ
    SStranspose = collect(S * S')
    p1, p2, λ1, λ2 = # ...
    P = # ...
    Σ = # ...
    return P, Σ
end

P, Σ = get_PΣ(S)
println("Erreur entre S Sᵗ et P Σ^2 Pᵗ : ", norm(S * S' .- P * Σ * Σ * P'))
display(S)

5. __Étape 4__ : Complétez l'implémentation de la décomposition complète en calculant les matrices $U$ et $V$ via aux formules du cours. Vous pouvez coder à la main l'inversion de la matrice $S \in \mathbb{M}_{2,2}$. N'oubliez pas d'appliquer la permutation inverse à la fin de l'étape 1. Testez votre décomposition sur les exemples fournis.

In [ ]:
function my_decomposition(A::Matrix{Float64})::Tuple{Matrix{Float64},Matrix{Float64},Matrix{Float64}} # sorties : U, Σ, V
    Aperm, Π = permute_columns(A)
    Q, S, T = get_QST(Aperm)
    P, Σ = get_PΣ(S)

    Sm1 = # votre code pour l'inverse de S 
    U = Q * Sm1 * P * Σ
    V = Π * T * P
    return U, Σ, V
end

###### Banque de matrices test 
# A = [1.0 0.0 1.0 1.0 0.0; 0.0 1.0 0.0 1.0 1.0] 
# A = [0.0 1.0 0.0; 0.0 0.0 1.0; 0.0 0.0 0.1] 
# A = [0.0 0.0 0.0 1.0 0.5; 1.0 0.5 0.0 0.0 0.3] 
# A = [0.0 0.0 0.0 1.0 0.5; 0.0 0.5 0.0 0.0 0.3] 
function get_randtest()::Matrix{Float64}
    n = rand(2:20)
    m = rand(2:20)
    T1 = 3.0 .* (rand(n,2) .- 0.5)
    pos1 = rand(1:(m-1))
    pos2 = rand((pos1+1):m)
    A = zeros(n,m)
    A[:,pos1] = T1[:,1]
    A[:,pos2] = T1[:,2]
    for i in 1:m
        if i ∉ [pos1,pos2]
            A[:,i] = T1 * rand(2)
        end
    end
    return A
end
A = get_randtest()

U, Σ, V = my_decomposition(A)
display(U)
display(Σ)
display(V)
println("Erreur entre A et U * Σ * Vᵗ : ", norm(A .- U * Σ * V'))